### mysql 연결

In [1]:
import pickle

file = open("/Users/kangyeolyoun/Desktop/pickle file/aws_mysql_db", "rb")
data = pickle.load(file)

In [22]:
import sys
import requests
import base64
import json
import logging
import pymysql



host = data[0]
port = int(data[1])
username = "kangyeol"
database = data[2]
password = data[3]

def main():
    
    try:
        conn = pymysql.connect(host, user=username, passwd=password, db=database,\
                             port=port, use_unicode=True, charset="utf8")
        cursor = conn.cursor()
    except:
        logging.error("could not connect to RDS")
        sys.exit(1)
        
    cursor.execute("SHOW TABLES")
    #fetchall 은 모든 데이터를 가지고 오겠다.
    print(cursor.fetchall())
    
    
    
    print("success")
    sys.exit(0)

In [23]:
main()

(('people',),)
success


SystemExit: 0

### Spotify 테이블 만들기 (artist, artist_genres)

<img src="./imgs/2_1.png" width="60%">

In [ ]:
# MYSQL 접속후 CREATE TABLE artists (id VARCHAR(255), name VARCHAR(255), followers INTEGER, popularity INTEGER, url VARCHAR(255), image_url VARCHAR(255), PRIMARY KEY(id)) ENGINE=InnoDB DEFAULT CHARSET="utf8";
# CREATE TABLE artists (id VARCHAR(255), name VARCHAR(255), followers INTEGER, popularity INTEGER, url VARCHAR(255), image_url VARCHAR(255), PRIMARY KEY(id)) ENGINE=InnoDB DEFAULT CHARSET="utf8";
# CREATE TABLE artist_genres (artist_id VARCHAR(255), genre VARCHAR(255)) ENGINE=InnoDB DEFAULT CHARSET="utf8";

# SHOW CREATE TABLE artists 


### artist_genres 에 data_insert

In [ ]:
# show create table artist_genres;
# INSERT INTO artist_genres (artist_id, genre) VALUES ('1234', 'pop');

# select * from artist_genres;


### 똑같은 데이터 insert 방지위해 unique key 설정

In [ ]:
# DROP TABLE artist_genres;
# CREATE TABLE artist_genres (artist_id VARCHAR(255), genre VARCHAR(255), UNIQUE KEY(artist_id, genre)) ENGINE=InnoDB DEFAULT CHARSET="utf8";

# ALTER TABLE artist_genres ADD COLUMN country VARCHAR(255);
#언제 업데이트가 되었는지 추가해줍니다.
# ALTER TABLE artist_genres ADD COLUMN updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP;

# REPLACE 의 경우 key값을 찾아서 지운 후 새로운것을 추가한다. #id 값이 변하기 떄문에... 주의해야한다.
# REPLACE INTO artist_genres (artist_id, genre, country) VALUES ("1234","pop", "UK");

# 가장 좋은 방법 (Insert INTO)
# INSERT INTO artist_genres (artist_id, genre, country) VALUES("1234", "rock", "FR") ON DUPLICATE KEY UPDATE artist_id="1234", genre="rock", country="FR";
# INSERT INTO artist_genres (artist_id, genre, country) VALUES("1234", "hip-hop", "FR") ON DUPLICATE KEY UPDATE artist_id="1234", genre="hip-hop", country="FR";


In [5]:
import sys
import requests
import base64
import json
import logging
import pymysql



host = data[0]
port = int(data[1])
username = "kangyeol"
database = data[2]
password = data[3]

def main():
    
    try:
        conn = pymysql.connect(host, user=username, passwd=password, db=database,\
                             port=port, use_unicode=True, charset="utf8")
        cursor = conn.cursor()
    except:
        logging.error("could not connect to RDS")
        sys.exit(1)
        
    cursor.execute("SHOW TABLES")
    #fetchall 은 모든 데이터를 가지고 오겠다.
    print(cursor.fetchall())
    
    #%s 는 string을 의미
    query = 'INSERT INTO artist_genres (artist_id, genre) \
    VALUES ("%s", "%s")' %("2345", 'pop')
    cursor.execute(query)
    conn.commit()
    

    sys.exit(0)

In [6]:
main()

(('artist_genres',), ('artists',), ('people',))


SystemExit: 0

In [7]:
import sys
import requests
import base64
import json
import logging
import pymysql



host = data[0]
port = int(data[1])
username = "kangyeol"
database = data[2]
password = data[3]

def main():
    
    try:
        conn = pymysql.connect(host, user=username, passwd=password, db=database,\
                             port=port, use_unicode=True, charset="utf8")
        cursor = conn.cursor()
    except:
        logging.error("could not connect to RDS")
        sys.exit(1)
        
    cursor.execute("SHOW TABLES")
    #fetchall 은 모든 데이터를 가지고 오겠다.
    print(cursor.fetchall())
    
    # .format
    query = 'INSERT INTO artist_genres (artist_id, genre) \
    VALUES ("{}", "{}")'.format("2345", "hip-hop")
    cursor.execute(query)
    conn.commit()
    

    sys.exit(0)

In [8]:
main()

(('artist_genres',), ('artists',), ('people',))


SystemExit: 0

### Dictionary, JSON Package

In [38]:
def get_headers(client_id, client_secret):

    endpoint = "https://accounts.spotify.com/api/token"
    encoded = base64.b64encode("{}:{}".format(client_id, client_secret).encode('utf-8')).decode('ascii')

    headers = {
        "Authorization": "Basic {}".format(encoded)
    }

    payload = {
        "grant_type": "client_credentials"
    }

    r = requests.post(endpoint, data=payload, headers=headers)

    access_token = json.loads(r.text)['access_token']

    headers = {
        "Authorization": "Bearer {}".format(access_token)
    }

    return headers

In [47]:
import sys
import requests
import base64
import json
import logging
import pymysql

host = data[0]
port = int(data[1])
username = "kangyeol"
database = data[2]
password = data[3]
client_id = data[4]
client_secret = data[5]


def main():
    
    
    
    try:
        conn = pymysql.connect(host, user=username, passwd=password, db=database,\
                             port=port, use_unicode=True, charset="utf8")
        cursor = conn.cursor()
    except:
        logging.error("could not connect to RDS")
        sys.exit(1)
        
    headers = get_headers(client_id, client_secret)
    
    ## Spotify Search API
    params = {
        "q": "BTS",
        "type": "artist",
        "limit": "5"
    }
    
    r = requests.get("https://api.spotify.com/v1/search", params=params, headers=headers)
    
    # json을 통해서 스트링 형태로 들어온다.
    raw = json.loads(r.text)
    print(raw["artists"])
    sys.exit(0)
    
    


In [48]:
main()

{'href': 'https://api.spotify.com/v1/search?query=BTS&type=artist&offset=0&limit=5', 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3Nrfpe0tUJi4K4DXYWgMUX'}, 'followers': {'href': None, 'total': 16226414}, 'genres': ['k-pop', 'k-pop boy group'], 'href': 'https://api.spotify.com/v1/artists/3Nrfpe0tUJi4K4DXYWgMUX', 'id': '3Nrfpe0tUJi4K4DXYWgMUX', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/0c9057cb30520f9f883a220051260fc66a2f3ffa', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/34ee854082d0ea4d571e2ecf7f4aecea61270ad0', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/156147697b696d4a6180e037dc1e2a33117d8d4a', 'width': 160}], 'name': 'BTS', 'popularity': 94, 'type': 'artist', 'uri': 'spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/51sg5jUqKu2tkbmPlwPrNH'}, 'followers': {'href': None, 'total': 137493}, 'genres': [], 'href': 'https://api.spotify.com/v1/artists/51sg

SystemExit: 0

In [65]:
import sys
import requests
import base64
import json
import logging
import pymysql

host = data[0]
port = int(data[1])
username = "kangyeol"
database = data[2]
password = data[3]
client_id = data[4]
client_secret = data[5]


def main():
    
    
    
    try:
        conn = pymysql.connect(host, user=username, passwd=password, db=database,\
                             port=port, use_unicode=True, charset="utf8")
        cursor = conn.cursor()
    except:
        logging.error("could not connect to RDS")
        sys.exit(1)
        
    headers = get_headers(client_id, client_secret)
    
    ## Spotify Search API
    params = {
        "q": "BTS",
        "type": "artist",
        "limit": "5"
    }
    
    r = requests.get("https://api.spotify.com/v1/search", params=params, headers=headers)
    
    # json을 통해서 스트링 형태로 들어온다.
    raw = json.loads(r.text)
    print(raw["artists"].keys())
    
    print(raw["artists"]["items"])
    
    {
        "id",
        "name",
        "followers"
        "popularirty"
        "url"
        "image_url"
    }
    
    sys.exit(0)
    
    


In [66]:
# keys만 검색하여서 우리가 필요한 정보는 items에 있다는것을 알 수 있다.
main()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])
[{'external_urls': {'spotify': 'https://open.spotify.com/artist/3Nrfpe0tUJi4K4DXYWgMUX'}, 'followers': {'href': None, 'total': 16226414}, 'genres': ['k-pop', 'k-pop boy group'], 'href': 'https://api.spotify.com/v1/artists/3Nrfpe0tUJi4K4DXYWgMUX', 'id': '3Nrfpe0tUJi4K4DXYWgMUX', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/0c9057cb30520f9f883a220051260fc66a2f3ffa', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/34ee854082d0ea4d571e2ecf7f4aecea61270ad0', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/156147697b696d4a6180e037dc1e2a33117d8d4a', 'width': 160}], 'name': 'BTS', 'popularity': 94, 'type': 'artist', 'uri': 'spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/51sg5jUqKu2tkbmPlwPrNH'}, 'followers': {'href': None, 'total': 137493}, 'genres': [], 'href': 'https://api.spotify.com/v1/artists/51sg5jUqKu2tkbmPlwPrN

SystemExit: 0

In [92]:
# 원하는 데이터 뽑아오자
import sys
import requests
import base64
import json
import logging
import pymysql

host = data[0]
port = int(data[1])
username = "kangyeol"
database = data[2]
password = data[3]
client_id = data[4]
client_secret = data[5]


def main():
    
    
    
    try:
        conn = pymysql.connect(host, user=username, passwd=password, db=database,\
                             port=port, use_unicode=True, charset="utf8")
        cursor = conn.cursor()
    except:
        logging.error("could not connect to RDS")
        sys.exit(1)
        
    headers = get_headers(client_id, client_secret)
    
    ## Spotify Search API
    params = {
        "q": "BTS",
        "type": "artist",
        "limit": "5"
    }
    
    r = requests.get("https://api.spotify.com/v1/search", params=params, headers=headers)
    
    # json을 통해서 스트링 형태로 들어온다.
    raw = json.loads(r.text)

    
    artist_raw = raw["artists"]["items"][0]
    
    if artist_raw["name"] == params["q"]:
        artist = {
            "id": artist_raw["id"],
            "name": artist_raw["name"],
            "followers": artist_raw["followers"]["total"],
            "popularity": artist_raw["popularity"],
            "url": artist_raw["external_urls"]["spotify"],
            "image_url": artist_raw["images"][0]["url"]
        }
    

        
    query = """
        INSERT INTO artists (id, name, followers, popularity, url, image_url)
        VALUES ("{}", "{}", {}, {}, "{}", "{}")
        ON DUPLICATE KEY UPDATE id= "{}", name= "{}", followers={},\
        popularity={}, url="{}", image_url="{}"
    """.format(
        artist["id"],
        artist["name"],
        artist["followers"],
        artist["popularity"],
        artist["url"],
        artist["image_url"],
        artist["id"],
        artist["name"],
        artist["followers"],
        artist["popularity"],
        artist["url"],
        artist["image_url"]
    )
        
    print(query)
        

    cursor.execute(query)
    conn.commit()
    sys.exit(0)
    
    


In [93]:
 main()


        INSERT INTO artists (id, name, followers, popularity, url, image_url)
        VALUES ("3Nrfpe0tUJi4K4DXYWgMUX", "BTS", 16226414, 94, "https://open.spotify.com/artist/3Nrfpe0tUJi4K4DXYWgMUX", "https://i.scdn.co/image/0c9057cb30520f9f883a220051260fc66a2f3ffa")
        ON DUPLICATE KEY UPDATE id= "3Nrfpe0tUJi4K4DXYWgMUX", name= "BTS", followers=16226414,        popularity=94, url="https://open.spotify.com/artist/3Nrfpe0tUJi4K4DXYWgMUX", image_url="https://i.scdn.co/image/0c9057cb30520f9f883a220051260fc66a2f3ffa"
    


SystemExit: 0